# IE582- Final


## Ahmet Buğra Taksuk - 2017402168

### Introduction

- In this task, we are asked to classify users who request loans from the bank as good or default based on their information encodded in bank's database. For the evaluation based, we are also asked to evalueate our models on a cost matrix appraoch where we lose %15 our money if we missclassify a good user and all the money if we missclassify a default user. For descriptive analysis purposes, statistics of sample, missing value  analysis, adding dummy columns and checking unique values are steps I followed in this step. To attack the problem, I performed Repeated Cross Validation on different classifiers by tuning their parameters and check their results via error function I pass the classifiers. 

### Descriptive Analysis

In [1]:
# Necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from scipy.spatial import distance
sns.set(rc={'figure.figsize':(8,6)})

In [600]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [601]:
train

,loan_application_id,loan_amount,default,customer_age,Var_1,Var_2,Var_3,Var_4,Var_5,Var_6,Var_7,Var_8,Var_9,Var_10,Var_11,Var_12,Var_13,Var_14,Var_15,Var_16,Var_17,Var_18,Var_19,Var_20,Var_21,Var_22,Var_23,Var_24,Var_25,Var_26,Var_27,Var_28,Var_29,Var_30,Var_31,Var_32,Var_33,Var_34,Var_35,Var_36,Var_37,Var_38,Var_39,Var_40,Var_41,Var_42,Var_43,Var_44,Var_45,Var_46,Var_47,Var_48,Var_49,Var_50,Var_51,Var_52,Var_53,Var_54,Var_55,Var_56,Var_57,Var_58,Var_59
0,1,2855.0,0,25,3,1,1,37,41,5,5,1330,1,4,0,0,0,17,0,0,13000,1,4,1,4,600.0,0,0,600.0,600,37,1,465,0,600,0,0.00,0,0,0,0,0,N,0.00,0,0,0,0,0,0,0.00,0,0,0,0,0,N,0.0,0,0,1,1,0
1,2,1577.0,0,53,0,0,0,0,26,2,2,1478,1,1,0,0,0,0,0,0,2196,1,1,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,26.00,203,4300,1,3718,4300,N,0.61,203,4300,3718,203,29,4300,26.00,0,0,0,0,0,N,0.0,0,0,0,1,0
2,3,6858.0,0,39,4,2,3,55,58,10,10,1060,3,4,0,0,0,0,0,0,5000,3,4,4,4,1500.0,0,2001,2500.0,2000,55,4,4999,0,10000,2001,13.42,521,14000,1,4120,14000,N,0.35,521,14000,4120,521,36,14000,13.42,521,521,700,0,700,N,700.0,0,0,4,6,0
3,4,2434.5,0,49,3,1,3,98,98,11,11,1218,4,7,0,0,0,92,2000,1,7100,4,6,6,7,2450.0,0,0,2450.0,3900,98,2,0,0,4900,15,0.65,296,8100,1,0,8100,N,0.00,296,8100,0,296,36,8100,0.65,296,296,16350,127,16935,N,8467.5,0,0,0,5,0
4,5,4964.0,0,47,2,1,1,110,124,15,15,1334,4,6,0,0,0,105,5250,2,3200,4,4,3,6,4100.0,700,0,4100.0,4100,110,1,3116,552,4100,700,21.43,1082,30000,7,38044,81941,N,0.90,1082,30000,38044,2684,225,81941,21.43,1874,2684,3000,142,3400,N,1700.0,0,0,2,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,2556,3931.0,0,47,3,2,2,64,64,18,18,1003,10,13,0,0,0,64,15000,3,28190,10,8,11,13,1200.0,450,0,1200.0,1200,13,1,1192,252,1200,450,3.09,967,33000,3,1726,35094,N,0.74,967,33000,1726,1125,77,35094,3.09,1125,1125,500,491,500,N,500.0,0,0,2,5,0
1711,2559,1995.0,0,64,3,0,0,127,127,17,17,1406,9,14,0,0,0,41,19500,6,18000,9,6,5,14,1500.0,1250,0,1500.0,1500,127,1,1539,0,1500,1250,7.12,260,8500,2,1897,12000,Y,0.26,260,8500,1897,456,69,12000,7.12,456,456,300,0,300,N,300.0,10103,456,0,4,0
1712,2560,3821.0,0,25,3,0,2,45,45,6,6,1122,1,2,0,0,0,45,0,0,5000,1,2,0,2,1100.0,1914,0,1475.0,1100,19,2,2848,73,2950,1914,3.13,411,10000,1,702,10000,N,0.77,411,10000,702,411,36,10000,3.13,411,411,350,347,350,N,350.0,0,0,0,4,0
1713,2561,3570.0,1,21,1,4,9,27,27,1,1,1218,0,0,0,0,0,0,0,0,0,0,0,0,0,1100.0,40,0,1100.0,1100,27,1,1098,0,1100,40,0.00,0,0,0,0,0,N,0.00,0,0,0,0,0,0,0.00,0,0,0,0,0,N,0.0,0,0,0,1,0


- Summary Statistics

In [602]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_application_id,1715.0,1273.198251,749.022598,1.0,617.50,1263.0,1934.0,2562.0
loan_amount,1715.0,2949.093644,1323.389125,771.0,1968.75,2677.0,3763.0,7994.0
default,1715.0,0.100292,0.300476,0.0,0.00,0.0,0.0,1.0
customer_age,1715.0,41.323032,13.669056,19.0,29.00,40.0,52.0,74.0
Var_1,1715.0,2.433236,1.517903,0.0,1.00,2.0,3.0,9.0
...,...,...,...,...,...,...,...,...
Var_55,1715.0,1977.721866,6323.798786,0.0,0.00,0.0,0.0,63535.0
Var_56,1715.0,106.966764,394.978591,0.0,0.00,0.0,0.0,11957.0
Var_57,1715.0,1.040816,1.278843,0.0,0.00,1.0,2.0,9.0
Var_58,1715.0,4.439650,2.890505,0.0,3.00,4.0,6.0,27.0


- Types of variables

In [603]:
train.dtypes.value_counts()

int64      54
float64     7
object      2
dtype: int64

- Missing Value Check

In [604]:
train.isna().sum()

loan_application_id    0
loan_amount            0
default                0
customer_age           0
Var_1                  0
                      ..
Var_55                 0
Var_56                 0
Var_57                 0
Var_58                 0
Var_59                 0
Length: 63, dtype: int64

- Unique Value Check
- All applications are unique so this feature can be used as user id 

In [605]:
len(train['loan_application_id'].unique())

1715

- Imbalance Dataset Check 
- So the data is imbalanced we need to pass the model class_weight

In [606]:
train['default'].value_counts()

0    1543
1     172
Name: default, dtype: int64

-  Mutual Information analysis for correlation between variables and target 

- No significant relationship is observed

In [607]:



from sklearn.feature_selection import mutual_info_classif
MI = pd.DataFrame(data = mutual_info_classif(train.drop(columns = ['default', 'Var_53','Var_39']),
                                        train['default']).reshape(1,-1) ,
                                        columns = train.drop(columns = ['default', 'Var_53','Var_39']).columns)

In [608]:
MI

,loan_application_id,loan_amount,customer_age,Var_1,Var_2,Var_3,Var_4,Var_5,Var_6,Var_7,Var_8,Var_9,Var_10,Var_11,Var_12,Var_13,Var_14,Var_15,Var_16,Var_17,Var_18,Var_19,Var_20,Var_21,Var_22,Var_23,Var_24,Var_25,Var_26,Var_27,Var_28,Var_29,Var_30,Var_31,Var_32,Var_33,Var_34,Var_35,Var_36,Var_37,Var_38,Var_40,Var_41,Var_42,Var_43,Var_44,Var_45,Var_46,Var_47,Var_48,Var_49,Var_50,Var_51,Var_52,Var_54,Var_55,Var_56,Var_57,Var_58,Var_59
0,0.0,0.011126,0.012601,0.004551,0.021485,0.029972,0.003189,0.009373,0.013107,0.0,0.029955,0.0,0.003903,0.0,0.010956,0.0,0.0,0.0,0.0,0.008367,0.00143,0.0,0.005608,0.0,0.0,0.0,0.018338,0.002543,0.010052,0.015907,0.0,0.0,0.002697,0.000545,0.000324,0.003117,0.0,0.00977,0.006287,0.012093,0.000813,0.0,0.0,0.008052,0.012334,0.0,0.00968,0.006582,0.004229,0.000062,0.0,0.018975,0.0,0.0,0.004355,0.0,0.0,0.0,0.000774,0.0


- For the 'Y' 'N' questions, dummy columns are added. 

In [609]:
train = pd.get_dummies(train, columns = ['Var_53','Var_39'],drop_first = True)

In [610]:
train.dtypes.value_counts()

int64      54
float64     7
uint8       2
dtype: int64

### Approach
- For this section, I am going to try different models by tuning their parameters and decide which model is the best in terms of the financial loss error resulted in Repeated Cross Validation. For parameter tuning, for example fpr the decision tree classifier, different models with various depth and number of trees are compared according to their CV results and then the best model tested on training data itself. For the model selection, I am going to try three different models which are Random Forests , Penazed Logistic Regression and XGBoost. The reason why I chose Random Forest is it generally works fine with when the number of columns are realtively high. The reason why I chose penalized regression it might be helpful since we have many features some might be irrelevant. And lastly, the reason why I chose XGBoost is that it uses boosting and tree based so it can benefit from a lot of features.

In [611]:
overall_scores = []

#### Defining Custom Loss Score based on penalites 

In [612]:
def financial_loss_scorer(y_train, y_pred,**kwargs):
    
    data = kwargs['data']
    data = pd.concat([data,y_train,pd.Series(y_pred)],axis = 1)

    index_1 = np.where((y_train == 0) & (y_pred == 1))
    index_2 = np.where((y_train == 1) & (y_pred == 0))

    loss_1 = np.sum(data['loan_amount'][index_1[0]]) * 0.15
    loss_2 = np.sum(data['loan_amount'][index_2[0]])
    return loss_1 + loss_2 


In [613]:
from sklearn.metrics import make_scorer
custom_scorer = make_scorer(financial_loss_scorer, data=X_train, greater_is_better=False)


#### Parameter Tuning For Random Forest 

In [564]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

X_train = train.drop(columns = ['default','loan_application_id'])
y_train = train['default']
scores = []
Depth_vals = [2,3,4,5,6]
trees = [100,200,300,400]

for i in Depth_vals:
    for j in trees:
        # prepare the cross-validation procedure
        cv = RepeatedKFold(n_splits=10, n_repeats=5 ,random_state=0)
        # create model
        model = RandomForestClassifier(max_depth = i,n_estimators = j, random_state = 0,  class_weight = 'balanced')
        # evaluate model
        scores.append(np.mean(cross_val_score(model, X_train, y_train, scoring=custom_scorer, cv=cv)))
        # report performance

print("Best Depth: ", Depth_vals[np.argmax(scores)], "Best #of Tree: ", trees[np.argmax(scores)],
          "Score:",  scores[np.argmax(scores)])

Best Depth:  3 Best #of Tree:  200 Score: -36766.5361


#### Performance of Random Forest On Train Data

In [565]:
from sklearn.ensemble import RandomForestClassifier

overall_scores.append(['Random Forest CV',scores[np.argmax(scores)]])

X_train = train.drop(columns = ['default','loan_application_id'])
y_train = train['default']
model = RandomForestClassifier(max_depth = Depth_vals[np.argmax(scores)],n_estimators = trees[np.argmax(scores)],random_state = 0,  class_weight = 'balanced')
model.fit(X_train,y_train)
overall_scores.append(['Random Forest Model Score',custom_scorer(model, X_train,y_train)])
print(f'Financial Loss Error with Random Forest Depth = {Depth_vals[np.argmax(scores)]} is ', custom_scorer(model, X_train,y_train))

Financial Loss Error with Random Forest Depth = 3 is  -267267.255


#### Paramater Tuning for Penalized Logistic Regression

In [589]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

X_train = train.drop(columns = ['default','loan_application_id'])
y_train = train['default']
scores = []
penalties = [2,3,4,5,6,7,8,9,10]

for i in penalties:
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=5 ,random_state=0)
    # create model
    model = LogisticRegression(penalty = 'l1', C = i, class_weight = 'balanced',  solver = 'saga', random_state=0, max_iter = 10000)
    # evaluate model
    scores.append(np.mean(cross_val_score(model, X_train, y_train, scoring=custom_scorer, cv=cv)))
    # report performance

print("Best Penalty Term: ", penalties[np.argmax(scores)], 
          "Score:",  scores[np.argmax(scores)])

Best Penalty Term:  2 Score: -39863.3364


#### Performance of Penalized Regression On Train Data

In [591]:
from sklearn.linear_model import LogisticRegression

overall_scores.append(['Penalized Logistic Regression',scores[np.argmax(scores)]])

X_train = train.drop(columns = ['default','loan_application_id'])
y_train = train['default']
model = model = LogisticRegression(penalty = 'l1', C = penalties[np.argmax(scores)], class_weight = 'balanced',  solver = 'saga', random_state=0, max_iter = 10000)
model.fit(X_train,y_train)
overall_scores.append(['Penalized Logistic Regression Model Score',custom_scorer(model, X_train,y_train)])
print(f'Financial Loss Error with Logistic Regression with Penalty = {penalties[np.argmax(scores)]} is ', custom_scorer(model, X_train,y_train))

Financial Loss Error with Logistic Regression with Penalty = 2 is  -373308.755


#### Paramater Tuning for XGBoost Classifier 

In [623]:
import xgboost  as xgb
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV



scores = []

model = xgb.XGBClassifier(use_label_encoder=False, random_state=0,
                          eval_metric='error')

parameters = {'objective': ["reg:logistic"],
              'max_depth': [2, 4, 6], 
              'eta': [0.1],
              "scale_pos_weight": [8],
              "n_estimators" :[100, 200, 300]}

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=5 ,random_state=0)
# create model
searcher = GridSearchCV(estimator=model, scoring = custom_scorer,
                         cv=cv, param_grid= parameters)

grid_result = searcher.fit(X_train, y_train)


# evaluate model
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -36772.425900 using {'eta': 0.1, 'max_depth': 2, 'n_estimators': 100, 'objective': 'reg:logistic', 'scale_pos_weight': 8}


In [626]:
overall_scores.append(['XGB Model Performance',grid_result.best_score_])


model = xgb.XGBClassifier(use_label_encoder=False, random_state=0,
                          eval_metric='error', eta = 0.1, max_depth = 2, n_estimators = 100, objective = 'reg:logistic', scale_pos_weight = 8)
model.fit(X_train,y_train)
overall_scores.append(['XGB  Model Score',custom_scorer(model, X_train,y_train)])
print(f'Financial Loss Error with XGB Model  is ', custom_scorer(model, X_train,y_train))

Financial Loss Error with XGB Model  is  -193928.055


### Evaluation
- By looking at the results, the best score we have is with XGBoost, therefore we are going to predict with that model. Overall model performances and Cross Validation performances can be seen below also. Since XGBoost uses boosting methods and it is tree based, it is reasonable that we got the best score with that beacuse trees are better with when we have more feature.  

- Random Forest with  Depth:  3  #of Tree:  200 CV Score: -36766.5361
- Financial Loss Error with Random Forest Depth = 3 is  -267267.255
- Penalized Logistic Regression with Penalty Term:  2 CV Score: -39863.3364
- Financial Loss Error with Logistic Regression with Penalty = 2 is  -373308.755
- XGBoost with 'eta': 0.1, 'max_depth': 2, 'n_estimators': 100 CV Score : -36772.425900
- Financial Loss Error with XGB Model  is  -193928.055

#### Prediction 

In [630]:
test = pd.get_dummies(test, columns = ['Var_53','Var_39'],drop_first = True)
test.set_index('loan_application_id', inplace = True)

,loan_amount,default,customer_age,Var_1,Var_2,Var_3,Var_4,Var_5,Var_6,Var_7,Var_8,Var_9,Var_10,Var_11,Var_12,Var_13,Var_14,Var_15,Var_16,Var_17,Var_18,Var_19,Var_20,Var_21,Var_22,Var_23,Var_24,Var_25,Var_26,Var_27,Var_28,Var_29,Var_30,Var_31,Var_32,Var_33,Var_34,Var_35,Var_36,Var_37,Var_38,Var_40,Var_41,Var_42,Var_43,Var_44,Var_45,Var_46,Var_47,Var_48,Var_49,Var_50,Var_51,Var_52,Var_54,Var_55,Var_56,Var_57,Var_58,Var_59,Var_53_Y,Var_39_Y
loan_application_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,2106.0,NaN,58,2,0,6,101,101,20,21,1184,13,16,0,0,0,101,18000,3,43000,13,12,12,16,0.00,0,0,450.00,0,1,1,0,0,450,0,1.00,1292,45000,1,889,45000,0.71,1292,45000,889,1292,48,45000,1.00,1292,1292,1500,1533,3300,1100.0,0,0,1,5,0,0,0
9,4034.0,NaN,49,2,2,3,53,76,15,15,987,5,9,0,0,0,53,7400,2,7839,5,4,7,9,1100.00,1130,0,1100.00,1100,26,1,1129,0,1100,1130,10.60,30,827,4,515,1907,0.94,30,827,515,75,132,1907,10.60,75,75,500,141,650,325.0,0,0,4,7,0,0,1
18,2742.0,NaN,64,2,0,1,54,54,5,5,1198,0,0,0,0,0,0,0,0,0,0,0,0,0,1250.00,750,0,1250.00,1250,54,2,1343,1005,2500,750,2.23,183,6000,2,271,9000,0.58,183,6000,271,294,84,9000,2.23,294,294,500,0,500,500.0,8729,294,0,5,0,0,0
20,1752.0,NaN,69,4,1,2,140,140,12,12,1622,3,4,0,0,0,85,5200,1,11000,3,3,3,4,1766.67,0,0,1766.67,3500,140,3,0,0,5300,87,9.69,607,12000,4,7652,17630,0.61,607,12000,7652,1094,64,17630,9.69,607,607,2250,0,2250,2250.0,6075,607,0,8,0,0,0
22,2340.5,NaN,31,1,1,1,28,87,4,4,1305,1,2,0,0,0,0,0,0,26000,1,1,2,2,1000.00,0,0,1000.00,1000,28,1,0,0,1000,683,0.00,0,0,0,0,0,0.84,1138,82500,11084,1138,120,82500,26.81,0,1138,0,0,0,0.0,0,0,0,2,0,0,0


In [631]:
y_pred = model.predict(test.drop(columns = ['default']))
y_pred

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,

In [634]:
submission = pd.DataFrame(data = y_pred, index = test.index, columns = ['prediction'])
submission.head()

,prediction
loan_application_id,
6,0
9,1
18,0
20,0
22,0


In [635]:
submission.to_csv('test_predictions.csv')

### Conclusion
- As a result, we performed three different models by tuning their parameters and compared them according to their CV and performace measures. By looking at financial loss scores the best model we have is XGBoost which is expected since it uses boosting, it easier to divide the classification space into subspaces when there are a lot of features. Lastly, we predicted our test set with final model and write them into a CSV.